## 1. Model Visualization

In [ ]:
import pandas as pd

# Load original stock data (training + testing dataset)
stock_data = pd.read_parquet("./stock_data/Processed_Stock_Data.parquet")  # Ensure this is the correct dataset

# Identify the correct test start date
test_start_date = stock_data["Date"].iloc[-len(pd.read_parquet("./models/RandomForest_No_Indicators.parquet"))]  # Last N rows used for testing

print(f"Detected Test Start Date: {test_start_date}")

In [ ]:
# Load Random Forest Predictions
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")
lstm_with_ind_df = pd.read_parquet("./models/LSTM_With_Indicators_Predictions.parquet")

# Convert numerical index to datetime
rf_no_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(rf_no_ind_df), freq="B")  # Adjust start date
rf_with_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(rf_with_ind_df), freq="B")
lstm_no_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(lstm_no_ind_df), freq="B")
lstm_with_ind_df["Date"] = pd.date_range(start=test_start_date, periods=len(lstm_with_ind_df), freq="B")

# Save Updated Data
rf_no_ind_df.to_parquet("./models/RandomForest_No_Indicators.parquet", index=False)
rf_with_ind_df.to_parquet("./models/RandomForest_With_Indicators.parquet", index=False)
lstm_no_ind_df.to_parquet("./models/LSTM_No_Indicators_Predictions.parquet", index=False)
lstm_with_ind_df.to_parquet("./models/LSTM_With_Indicators_Predictions.parquet", index=False)

print("Fixed Date Formats for RF & LSTM")

In [ ]:
# Load all updated datasets
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")
lstm_with_ind_df = pd.read_parquet("./models/LSTM_With_Indicators_Predictions.parquet")

# Print first few dates to check alignment
print("Random Forest No Indicators Dates:", rf_no_ind_df["Date"].head())
print("Random Forest With Indicators Dates:", rf_with_ind_df["Date"].head())
print("LSTM No Indicators Dates:", lstm_no_ind_df["Date"].head())
print("LSTM With Indicators Dates:", lstm_with_ind_df["Date"].head())

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Load Predictions
rf_no_ind_df = pd.read_parquet("./models/RandomForest_No_Indicators.parquet")
rf_with_ind_df = pd.read_parquet("./models/RandomForest_With_Indicators.parquet")
lstm_no_ind_df = pd.read_parquet("./models/LSTM_No_Indicators_Predictions.parquet")
lstm_with_ind_df = pd.read_parquet("./models/LSTM_With_Indicators_Predictions.parquet")

# Load Actual Prices
actual_prices = rf_no_ind_df["Actual"]
test_dates = rf_no_ind_df["Date"]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add Actual Prices
fig.add_trace(go.Scatter(x=test_dates, y=actual_prices, mode='lines', name='Actual Price', line=dict(color='black')))

# Add Predictions
fig.add_trace(go.Scatter(x=test_dates, y=rf_no_ind_df["Predicted"], mode='lines', name='RF (No Indicators)', line=dict(dash='dot', color='blue')))
fig.add_trace(go.Scatter(x=test_dates, y=rf_with_ind_df["Predicted"], mode='lines', name='RF (With Indicators)', line=dict(dash='dot', color='red')))
fig.add_trace(go.Scatter(x=lstm_no_ind_df["Date"], y=lstm_no_ind_df["Predicted"], mode='lines', name='LSTM (No Indicators)', line=dict(dash='dash', color='green')))
fig.add_trace(go.Scatter(x=lstm_with_ind_df["Date"], y=lstm_with_ind_df["Predicted"], mode='lines', name='LSTM (With Indicators)', line=dict(dash='dash', color='orange')))

# Customize Layout
fig.update_layout(title="Stock Price Predictions: Actual vs Predicted",
                  xaxis_title="Date",
                  yaxis_title="Stock Price")

fig.show()

## 2. Indicators Visualization

In [ ]:
indicator_data = pd.read_parquet("./stock_data/Stock_Data_with_indicators.parquet")\

indicator_data.head(20)

In [ ]:
from plotly.subplots import make_subplots
# Ensure Date is in datetime format
indicator_data["Date"] = pd.to_datetime(indicator_data["Date"])

# Create subplots (4 rows: Stock Price, RSI, MACD, Moving Averages)
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.08, 
                    subplot_titles=("Stock Price with Moving Averages", "RSI with 80-20 Levels", "MACD with Signal Line"))

# 1️⃣ Stock Price with Moving Averages
fig.add_trace(go.Scatter(x=indicator_data["Date"], y=indicator_data["Close"], mode='lines', name="Stock Price", line=dict(color="black")), row=1, col=1)
fig.add_trace(go.Scatter(x=indicator_data["Date"], y=indicator_data["SMA_10"], mode='lines', name="SMA (10-day)", line=dict(color="red")), row=1, col=1)
fig.add_trace(go.Scatter(x=indicator_data["Date"], y=indicator_data["EMA_10"], mode='lines', name="EMA (10-day)", line=dict(color="green")), row=1, col=1)

# 2️⃣ RSI with 80-20 Levels
fig.add_trace(go.Scatter(x=indicator_data["Date"], y=indicator_data["RSI"], mode='lines', name="RSI", line=dict(color="blue")), row=2, col=1)
fig.add_hline(y=80, line=dict(color="red", dash="dot"), row=2, col=1)  # Overbought level
fig.add_hline(y=20, line=dict(color="green", dash="dot"), row=2, col=1)  # Oversold level

# 3️⃣ MACD with Volume Bars
fig.add_trace(go.Scatter(x=indicator_data["Date"], y=indicator_data["MACD"], mode='lines', name="MACD", line=dict(color="purple")), row=3, col=1)
fig.add_trace(go.Bar(x=indicator_data["Date"], y=indicator_data["Volume"], name="Volume", marker_color="green", opacity=0.4), row=3, col=1)

fig.update_layout(
    title="Technical Indicators Segregated with Time Range Filter",
    height=900,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1D", step="day", stepmode="backward"),
                dict(count=3, label="3D", step="day", stepmode="backward"),
                dict(count=7, label="1W", step="day", stepmode="backward"),
                dict(count=1, label="1M", step="month", stepmode="backward"),
                dict(count=6, label="6M", step="month", stepmode="backward"),
                dict(count=1, label="1Y", step="year", stepmode="backward"),
                dict(step="all", label="All")
            ])
        ),
        type="date"
    )
)

fig.show()